In [ ]:
!pip3 install geopandas
!pip3 install geopy
!pip3 install CurrencyConverter
!pip install transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.6/611.6 kB 11.3 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Verify that dataExtraction.py is in the same directory
!ls

# Add the current directory to Python's path
import sys
sys.path.append('/content/drive/MyDrive/Data240/')  # Adjust this path if the file is in a different folder

# Now import your custom module
from dataExtraction import extract

# Test the function
print("Module imported successfully!")


drive  sample_data
Module imported successfully!


In [ ]:
# Imports

# Geo Tools
import geopandas as gpd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

# NLP
import re
from functools import partial
from transformers import pipeline
from currency_converter import CurrencyConverter

# Data Manipulation & Visualisation
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import statistics
import os

# Own modules
from dataExtraction import extract


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Data240/Total_Merged_Data.csv')
print(df.shape)
df.head(10)

(4488, 8)


,Title,Company Name,Location,Via,Description,Job Highlights,Date Posted,Schedule Type
0,Data Scientist,Apple,"Austin, TX",Careers At Apple,"Summary\nPosted: May 3, 2024\n\nRole Number:20...","An expert in advanced analytics, you are passi...",NaN,Full-time
1,Data Scientist,Select Minds LLC,"Dallas, TX",LinkedIn,Benefits/Perks\n• Competitive Compensation\n• ...,The ideal candidate has an analytical mind and...,13 hours ago,Full-time
2,Principal Data Scientist,MD Anderson,"Houston, TX",MD Anderson - MD Anderson Cancer Center,The mission of The University of Texas M. D. A...,An in-depth understanding of machine learning ...,NaN,Full-time
3,Data Scientist - Causal Discovery,CPChem,"The Woodlands, TX",Careers At Chevron Phillips - Chevron Phillips...,"You may not realize it, but you’ve likely used...",This position is a great opportunity for a pro...,9 days ago,Full-time
4,Senior Data Scientist-Remote,Baker Hughes,"Houston, TX (+1 other)",Baker Hughes,Senior Data Scientist-Remote\n\nAre you passio...,Have a Masters Degree in Computer Science or “...,5 days ago,Full-time
5,Imaging Data Scientist - Digital Pathology,MD Anderson,"Houston, TX",MD Anderson - MD Anderson Cancer Center,"MIN: $103,000 MID: $129,000 MAX: $155,000\n\nT...",The ideal candidate will have at least 1 year ...,7 days ago,Full-time
6,Data Scientist Principal (REMOTE) – Life Insur...,USAA,"Plano, TX (+6 others)",USAA Careers,Why USAA?\nLet’s do something that really matt...,"Bachelor’s degree in mathematics, computer sci...",8 days ago,Full-time
7,"Senior Data Scientist, Performance Analytics",ENGIE,"Houston, TX",Engie.com,"What You Can ExpectThe Senior Data Scientist, ...",NaN,NaN,Full-time
8,Data Scientist – Power Markets modelling,TotalEnergies,"Houston, TX",LinkedIn,Profil du candidat\n\nYou are someone who poss...,You are university educated to at least master...,6 days ago,Full-time
9,Principal Data Scientist,HP Inc.,"Austin, TX",Jobs - Towards AI,HP's Digital and Transformation Organization (...,Strong communication skills are essential for ...,3 days ago,Full-time


In [ ]:
df['Job Highlights'] = df['Job Highlights'].replace(r'\n\n|\n•|\n|\\n|\\n•|•', '', regex=True)  # Remove new line char
df['Job Highlights'] = df['Job Highlights'].str.slice(2, -2)  # Remove [] and additional "" marks

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4488 entries, 0 to 4487
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Title           4488 non-null   object
 1   Company Name    4488 non-null   object
 2   Location        4483 non-null   object
 3   Via             4488 non-null   object
 4   Description     4488 non-null   object
 5   Job Highlights  4186 non-null   object
 6   Date Posted     2724 non-null   object
 7   Schedule Type   3537 non-null   object
dtypes: object(8)
memory usage: 280.6+ KB


In [ ]:
df = df.drop(['Job ID', 'Share Link'], axis=1)

In [ ]:
df[['Company Name', 'Location', 'Via', 'Date Posted', 'Schedule Type', 'Job Highlights', 'Description', 'Title']].isna().sum()


,0
Company Name,0
Location,5
Via,0
Date Posted,1764
Schedule Type,951
Job Highlights,302
Description,0
Title,0


## Natural Language Processing (NLP) model to extract salary information from a dataset containing job postings

In [ ]:
import torch
from transformers import pipeline

def extract_salary(df):
    # Load the question-answering model and move it to GPU if available
    device = 0 if torch.cuda.is_available() else -1  # 0 means GPU, -1 means CPU
    qa_model = pipeline("question-answering", model='distilbert-base-cased-distilled-squad', device=device)

    question = "What is the salary or salary range for the job?"

    # Define a function to apply the model on each row
    def find_salary(row):
        # Combine the relevant text fields while handling missing values
        context = (
            str(row.get('Job Highlights', '')) + ' ' +
            str(row.get('Title', '')) + ' ' +
            str(row.get('Description', ''))
        )

        # Skip if context is empty
        if not context.strip():
            return "Not available"

        try:
            # Apply the model to get the answer
            answer = qa_model(question=question, context=context)
            # Return the answer if score is above threshold, otherwise return "Not available"
            return answer['answer'] if answer['score'] >= 0.3 else "Not available"

        except Exception as e:
            print(f"Error processing row: {e}")
            return "Not available"

    # Apply the function across the dataframe
    df['salary'] = df.apply(find_salary, axis=1)

    return df

# Example usage
df = extract_salary(df)
df.to_csv('NLPextrated_data.csv')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [ ]:
df.head()

,Title,Company Name,Location,Via,Description,Job Highlights,Date Posted,Schedule Type,salary
0,Data Scientist,Apple,"Austin, TX",Careers At Apple,"Summary\nPosted: May 3, 2024\n\nRole Number:20...","expert in advanced analytics, you are passion...",NaN,Full-time,Not available
1,Data Scientist,Select Minds LLC,"Dallas, TX",LinkedIn,Benefits/Perks\n• Competitive Compensation\n• ...,e ideal candidate has an analytical mind and a...,13 hours ago,Full-time,Not available
2,Principal Data Scientist,MD Anderson,"Houston, TX",MD Anderson - MD Anderson Cancer Center,The mission of The University of Texas M. D. A...,in-depth understanding of machine learning al...,NaN,Full-time,Not available
3,Data Scientist - Causal Discovery,CPChem,"The Woodlands, TX",Careers At Chevron Phillips - Chevron Phillips...,"You may not realize it, but you’ve likely used...",is position is a great opportunity for a profe...,9 days ago,Full-time,Not available
4,Senior Data Scientist-Remote,Baker Hughes,"Houston, TX (+1 other)",Baker Hughes,Senior Data Scientist-Remote\n\nAre you passio...,ve a Masters Degree in Computer Science or “ST...,5 days ago,Full-time,"between $104,000 - $150,000 USD"


## Cleaning and normalizing salary values in a DataFrame by handling salary notations (e.g., $100k, 1b USD) and converting them into a consistent numerical format for further processing or analysis.

In [ ]:
import pandas as pd
import re

# Define a function to expand salary values with "k" or "b" to full numerical values
def expand_salary_units(salary):
    # Convert to lowercase for consistent matching
    salary = salary.lower()

    # Replace "k" with "000" and "b" with "000000"
    salary = re.sub(r'(\d+)k\b', r'\1000', salary)  # Expand 100k to 100000
    salary = re.sub(r'(\d+)b\b', r'\1000000', salary)  # Expand 1b to 1000000

    return salary

# Define a function to clean unwanted symbols
def remove_unwanted_symbols(salary):
    salary = re.sub(r'[$,]', '', str(salary))  # Remove $ and commas for easier numerical parsing
    salary = re.sub(r'\bUSD\b|\bper year\b|\bper hour\b|\bannually\b|\bannual\b', '', salary, flags=re.IGNORECASE)
    salary = re.sub(r'[^\w\s.-]', '', salary)  # Remove any other non-alphanumeric symbols except dots and dashes
    return salary.strip()



# Apply the cleaning and expansion functions
df['Cleaned Salary'] = df['salary'].apply(remove_unwanted_symbols)
df['Expanded Salary'] = df['Cleaned Salary'].apply(expand_salary_units)

# Display the cleaned and expanded data
print(df[['salary', 'Cleaned Salary', 'Expanded Salary']].head())


                            salary           Cleaned Salary  \
0                    Not available            Not available   
1                    Not available            Not available   
2                    Not available            Not available   
3                    Not available            Not available   
4  between $104,000 - $150,000 USD  between 104000 - 150000   

           Expanded Salary  
0            not available  
1            not available  
2            not available  
3            not available  
4  between 104000 - 150000  


In [ ]:
df.head()

,Title,Company Name,Location,Via,Description,Job Highlights,Date Posted,Schedule Type,salary,Cleaned Salary,Expanded Salary,Min Salary,Max Salary
0,Data Scientist,Apple,"Austin, TX",Careers At Apple,"Summary\nPosted: May 3, 2024\n\nRole Number:20...","expert in advanced analytics, you are passion...",NaN,Full-time,Not available,Not available,not available,NaN,NaN
1,Data Scientist,Select Minds LLC,"Dallas, TX",LinkedIn,Benefits/Perks\n• Competitive Compensation\n• ...,e ideal candidate has an analytical mind and a...,13 hours ago,Full-time,Not available,Not available,not available,NaN,NaN
2,Principal Data Scientist,MD Anderson,"Houston, TX",MD Anderson - MD Anderson Cancer Center,The mission of The University of Texas M. D. A...,in-depth understanding of machine learning al...,NaN,Full-time,Not available,Not available,not available,NaN,NaN
3,Data Scientist - Causal Discovery,CPChem,"The Woodlands, TX",Careers At Chevron Phillips - Chevron Phillips...,"You may not realize it, but you’ve likely used...",is position is a great opportunity for a profe...,9 days ago,Full-time,Not available,Not available,not available,NaN,NaN
4,Senior Data Scientist-Remote,Baker Hughes,"Houston, TX (+1 other)",Baker Hughes,Senior Data Scientist-Remote\n\nAre you passio...,ve a Masters Degree in Computer Science or “ST...,5 days ago,Full-time,"between $104,000 - $150,000 USD",between 104000 - 150000,between 104000 - 150000,104000.0,150000.0


In [ ]:
df.to_csv('extracted_salary_data_total.csv')

## Handling salary data and converting hourly salary rates into equivalent annual salaries, on 40-hour work week and 52 weeks per year.

In [ ]:
import pandas as pd

# Load the dataset
file_path = '/content/drive/MyDrive/Data240/extracted_salary_data_total.csv'
data = pd.read_csv(file_path)

# Check for Min_Salary and Max_Salary columns
if 'Min Salary' in data.columns and 'Max Salary' in data.columns:
    # Define a threshold to identify hourly salary (e.g., values less than 200 are likely hourly rates)
    hourly_threshold = 200

    # Function to convert hourly to annual salary (assuming 40 hours per week and 52 weeks per year)
    def convert_hourly_to_annual(salary):
        return salary * 40 * 52

    # Convert Min_Salary and Max_Salary if they are below the threshold
    data['Min_Salary_Annual'] = data['Min Salary'].apply(
        lambda x: convert_hourly_to_annual(x) if x < hourly_threshold else x
    )
    data['Max_Salary_Annual'] = data['Max Salary'].apply(
        lambda x: convert_hourly_to_annual(x) if x < hourly_threshold else x
    )

    # Display the updated columns for verification
    print(data[['Min Salary', 'Max Salary', 'Min_Salary_Annual', 'Max_Salary_Annual']].head(10))

    # Save the updated dataset
    output_path = 'extracted_salary_data_with_annual.csv'
    data.to_csv(output_path, index=False)

    print(f"Updated dataset saved as: {output_path}")
else:
    print("The dataset does not contain the required 'Min_Salary' and 'Max_Salary' columns.")


   Min Salary  Max Salary  Min_Salary_Annual  Max_Salary_Annual
0         NaN         NaN                NaN                NaN
1         NaN         NaN                NaN                NaN
2         NaN         NaN                NaN                NaN
3         NaN         NaN                NaN                NaN
4    104000.0    150000.0           104000.0           150000.0
5         NaN         NaN                NaN                NaN
6    182680.0    349150.0           182680.0           349150.0
7     84000.0    128800.0            84000.0           128800.0
8         NaN         NaN                NaN                NaN
9         NaN         NaN                NaN                NaN
Updated dataset saved as: extracted_salary_data_with_annual.csv


In [ ]:
df = pd.read_csv('/content/extracted_salary_data_with_annual.csv')

In [ ]:
df.rename(columns={'Min_Salary_Annual': 'Min Salary'}, inplace=True)
df.rename(columns={'Max_Salary_Annual': 'Max Salary'}, inplace=True)

In [ ]:
df = df.drop(columns=['salary', 'Cleaned Salary', 'Expanded Salary'])

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Data240/processed_total_data.csv')


In [ ]:
df.head()

,Title,Company_Name,Location,Via,Min_Salary,Max_Salary,Programming_Languages,Visualization_Tools,Databases_and_Big_Data_Tools,ML_and_Data_Science_Libraries,Education,Experience,Soft_Skills,Work_Mode,job_position,level
0,Sr Data Analyst/ Data Modeler,Bright Vision Technologies,"Los Angeles, CA",LinkedIn,131494.897,163990.2606,sql,tableau,spark,"pandas, numpy, scikit-learn",bachelor,10,"problem-solving, communication skills",on-site,Data Analyst,Senior-level
1,TikTok Shop - Data Analyst Intern - 2025 Start,California Jobs,"Los Angeles, CA",Talentify,131494.897,163990.2606,"python, sql",tableau,mysql,tensorflow,bachelor,3,"teamwork, creativity, communication skills",hybrid,Data Analyst,Entry-level
2,Research Data Analyst (2-year fixed term),Stanford University,"La Cañada Flintridge, CA",Ladders,104358.000,128038.0000,sql,tableau,mysql,xgboost,bachelor,5,"presentation, interpersonal skills",hybrid,Data Analyst,Unspecified
3,Junior Data Analyst,SynergisticIT,"Pasadena, CA",LinkedIn,62025.000,82825.0000,"sql, python",tableau,mysql,tensorflow,bachelor,3,"problem-solving, communication",hybrid,Data Analyst,Entry-level
4,Research Data Analyst,Motion Recruitment,"Pasadena, CA",Motion Recruitment,161000.000,161000.0000,"r, python",tableau,mysql,xgboost,master,5,"problem-solving, presentation",hybrid,Data Analyst,Unspecified


## Extracting Latitude and Longitude for geospatial analysis

In [ ]:
df['Location'].value_counts()

,count
Location,
Anywhere,356
"New York, NY",251
United States,244
"Los Angeles, CA",103
"Denver, CO",68
...,...
"St Bernard, OH",1
"Woodbridge, IN",1
"Warrensville Heights, OH",1


In [ ]:
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import pandas as pd

In [ ]:
locations = df.Location.unique()  # Determine unique locations
locations_df = pd.DataFrame(data=locations, columns=['location'])  # Unique locations dataframe

geolocator = Nominatim(user_agent="data_madness")  # Initialize the geolocator to use Nominatim
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)  # Enforce a request delay to respect API

In [ ]:
def locator(x):
    if x['location'] == 'Anywhere':   # Job location listed as "Anywhere"
        x['address'] = "Not available"
        x['latitude'] = "Not available"
        x['longitude'] = "Not available"
        return x

    location = geolocator.geocode(x['location'], # Perform Geolocation search
                                  language='en',
                                  exactly_one=True)
    if location is not None:  # A location was found.
        x['address'] = location.address
        x['latitude'] = location.latitude
        x['longitude'] = location.longitude
    else:  # No location was found
        x['address'] = "Not available"
        x['latitude'] = "Not available"
        x['longitude'] = "Not available"
    return x

locations_df = locations_df.apply(locator, axis=1)
locations_df.head(10)

In [ ]:
df.to_csv('locations_cache.csv')